In [ ]:
import os
import dotenv

dotenv.load_dotenv()

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT"]

In [ ]:
from openai import AzureOpenAI


client = AzureOpenAI(
  azure_endpoint = endpoint, 
  api_key= api_key,  
  api_version="2024-08-01-preview"
)

In [ ]:
message_text = [
    {"role":"system","content":"Du bist ein AI Assistent der Menschen hilft Antworten zu finden."},
    {"role":"user","content":"Was ist die Telekom MMS?"},]

completion = client.chat.completions.create(
  model=deployment,
  messages = message_text,
)

print(completion.choices[0].message.content)

# RAG


RAG (Retrieval-Augmented Generation) ist ein Modell, das aus zwei Hauptkomponenten besteht: einem Retrieval-System und einem Generierungsmodell. Das Ziel von RAG ist es, die Qualität der Textgenerierung zu verbessern, indem es relevante und aktuelle Informationen aus einer großen Datenbank oder einem Datensatz abruft und diese Informationen zur Unterstützung des Generierungsprozesses verwendet.

Das Verfahren funktioniert in zwei Schritten:

1. Retrieval (Abruf): Zuerst sucht das Retrieval-System in einer umfangreichen Datenquelle nach Informationen, die relevant für die aktuelle Anfrage oder den Kontext sind. Diese Informationen werden als Unterstützung für die Textgenerierung ausgewählt.
2. Generation (Generierung): Anschließend verwendet das Generierungsmodell, in der Regel ein LLM, sowohl den ursprünglichen Text als auch die abgerufenen Informationen, um eine präzise und kontextuell relevante Antwort zu erzeugen.

In [ ]:
prompt = """Beantworten die Fragen der Nutzenden auf der Grundlage des unten stehenden Kontexts.

context:
Die Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht.
"""


message_text = [
    {"role":"system","content":prompt},
    {"role":"user","content":"Was ist die Telekom MMS?"},]

completion = client.chat.completions.create(
  model=deployment,
  messages = message_text,
)

print(completion.choices[0].message.content)

## Laden und durchsuchen von eigenen Daten


In [ ]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('./docs', glob="**/*.md")
data = loader.load()
len(data)

In [ ]:
# das array enthält alle texte aller Dateien

data[1]

### Wie können wir die Texte in kürzere Abschnitte unterteilen?

Web Demo [Chunk Visualizer](https://huggingface.co/spaces/m-ric/chunk_visualizer)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter,SentenceTransformersTokenTextSplitter,MarkdownHeaderTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=450, chunk_overlap=50)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))
all_splits

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
chroma = chromadb.Client()

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="ibm-granite/granite-embedding-278m-multilingual")
try:
    chroma.delete_collection("documents")
except:
    pass
collection = chroma.get_or_create_collection("documents",embedding_function=sentence_transformer_ef)

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=[item.page_content for item in all_splits], # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[item.metadata for item in all_splits], # filter on these!
    ids=[str(id) for id in range(0,len(all_splits))], # unique for each doc
)


In [ ]:
# Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["Wer ist der Schulungsleiter"],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    #where_document={"$contains":"Oliver"}  # optional filter
)

results

In [ ]:

def rag_chat(user_message, history):
    # 1. Abruf relevanter Dokumente (Retrieval)
    # Wir suchen in unserer Collection nach den 2 ähnlichsten Dokumenten
    results = collection.query(
        query_texts=[user_message],
        n_results=2
    )

    # 2. Aufbereitung des Kontexts
    # Die gefundenen Texte und Quellen werden lesbar zusammengestellt
    context = ""
    for doc, metadata in zip(results["documents"][0], results["metadatas"][0]):
        context += f"Quelle: {metadata['source']}\n"
        context += f"{doc}\n"
    
    # 3. Erstellen des Prompts (Generation)
    # Wir geben dem LLM die Anweisung, nur mit dem gefundenen Wissen zu antworten
    system_prompt = f"""Beantworte die Fragen der Nutzenden auf der Grundlage des Kontexts. Gib die Quelle der Informationen an.

    Context:
    {context}

    Wenn die Informationen zur Frage nicht im Kontext enthalten sind, dann antworte "Ich weiß es nicht."
    """

    # 4. Anfrage an das KI-Modell senden
    message_text = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]

    completion = client.chat.completions.create(
        model=deployment,
        messages=message_text
    )

    # Die Antwort des Modells zurückgeben
    return completion.choices[0].message.content



In [ ]:
rag_chat("Wer ist der Schulungsleiter?", [])

In [ ]:
import gradio as gr
# Erstellen der Benutzeroberfläche
# ChatInterface erstellt automatisch ein Chat-Fenster
demo = gr.ChatInterface(
    fn=rag_chat, 
    title="RAG Chatbot", 
    description="Ein einfacher Bot, der Fragen zu den geladenen Dokumenten beantwortet."
)

# Starten der Anwendung 
# (share=True erstellt öffentlichen Link, hier lassen wir es lokal)
demo.launch(server_port=7878)

## Nächster Schritt

Was passiert, wenn der Nutzer "Erzähle mir mehr!" im Chat eingibt? Wie können wir diese Feature implementieren?

Bonus: 

* Erstelle einen ChatBot für den EU AI Act. Welche Schritte sind dafür notwendig?

* Einstieg in [LangChain](https://www.langchain.com/). Die drei Notebooks im LangChain Ordner demonstrien wie man diesen Workflow mit Langchain implementiert.